# Test IE functions

In [ ]:
#| hide
from nbdev.showdoc import show_doc

%load_ext autoreload
%autoreload 2

In [ ]:
#| output: false
import numpy as np
import tempfile
from pandas import DataFrame
from pathlib import Path
import pandas as pd

from spannerflow.span import Span
from spannerlib.session import Session,test_session
from spannerlib.utils import assert_df_equals

In [ ]:
def yield_range_span(text: str,name:str):
    for i in range(len(text)):
        yield Span(text,0,i+1,name=name)

list(yield_range_span("hello","doc"))

In [ ]:
def yield_range(num: int):
        for i in range(num):
            yield i,

def yield_range_span(text: str,name:str):
    for i in range(len(text)):
        yield Span(text,0,i+1,name=name)
         
def yield_range_str(num: int):
    for i in range(num):
        yield "string" + str(i),


sess = test_session(
    [
        """
        test_range(X) <- yield_range(5) -> (X).
        ?test_range(X)
        """,

        """
        test_range_str(X) <- yield_range_str(5) -> (X).
        ?test_range_str(X)
        """,
        """
        test_range_raw(X) <- yield_range_span("hello","doc") -> (X).
        test_range_span(S) <- yield_range_span("hello","doc") -> (X), as_str(X)->(S).
        ?test_range_span(S)
        """,
    ],
    [
        pd.DataFrame({'X':range(5)}),
        pd.DataFrame({'X':["string0","string1","string2","string3","string4"]}),
        pd.DataFrame({'S':["h","he","hel","hell","hello"]})
    ],
    ie_funcs=[
        ['yield_range',yield_range,[int],[int]],
        ['yield_range_span',yield_range_span,[str,str],[Span]],
        ['yield_range_str',yield_range_str,[int],[str]]
    ],
)


In [ ]:

sess.export('?test_range_raw(S)',display_results=True)

In [ ]:
from spannerlib.ie_func.basic import rgx

In [ ]:
list(rgx('.*',"aa")) == ['aa',""]

In [ ]:
{(Span("aa",0,2),)} != {('aa',)}

In [ ]:
sess = test_session(
[
    """
    py_span_rel(X) <- rgx(".+","aa") -> (X).
    py_string_rel(X) <- rgx(".+","aa") -> (S), as_str(S)->(X).
    ?py_span_rel(X)
    """,
    """
    ?py_string_rel(X)
    """
],
[
    pd.DataFrame([
        ["aa"],
    ],columns=['X']),
    pd.DataFrame([
        ["aa"],
    ],columns=['X']),
],
)
# _ = sess.export('?py_span_rel(X)',display_results=True)

In [ ]:
sess = test_session(
[
    """
    py_span_rel(X,DEBUG) <- rgx(".+","aa") -> (X), print("X={0}, const={1}",X,"aa")->(DEBUG).
    ?py_span_rel(X,DEBUG)
    """,

],
[

    pd.DataFrame([
        ["aa","X=aa, const=aa"],
    ],columns=['X','DEBUG']),
],
)
# _ = sess.export('?py_span_rel(X)',display_results=True)

In [ ]:
# TODO add tests of extracting spans from spans once we have a way to treat spans as strings in the type system

In [ ]:
# uncomment these once we decide how to support rust regexs

In [ ]:
# test_session(
# [
#     """
#     string_rel(X) <- rgx_string("aa",".+") -> (X).
#     span_rel(X) <- rgx_span("aa",".+") -> (X).
#     ?string_rel(X)
#     """
#     """
#     ?span_rel(X)
#     """
# ],[
#     pd.DataFrame([
#         ['a'],
#         ['aa'],
#     ],columns=['X']),
#     pd.DataFrame([
#         [Span(0,1)],
#         [Span(0,2)],
#         [Span(1,2)]
#     ],columns=['X']),
# ],
# )

In [ ]:
# with tempfile.TemporaryDirectory() as temp_dir:
#         rgx_text_file = Path(temp_dir) / "temp"
#         with open(rgx_text_file, "w") as f:
#             f.write("aa")

# test_session(
# [
#     """
#     string_rel(X) <- rgx_string_from_file("tests/rgx_text_file",".+") -> (X).
#     span_rel(X) <- rgx_span_from_file("tests/rgx_text_file",".+") -> (X).
#     ?string_rel(X)
#     """
#     """
#     ?span_rel(X)
#     """
# ],[
#     pd.DataFrame([
#         ['a'],
#         ['aa'],
#     ],columns=['X']),
#     pd.DataFrame([
#         [Span(0,1)],
#         [Span(0,2)],
#         [Span(1,2)]
#     ],columns=['X']),
# ],
# )

In [ ]:
# text = "aab"
# pattern = "(?P<group_all>(?P<group_a>a+)(?P<group_b>b+))"
# test_session(
# [
#     f"""
#     group_string_rel(X,Y,Z) <- rgx_string("{text}","{pattern}") -> (X,Y,Z).
#     ?group_string_rel(X, Y, Z)
#     """
# ],[
#     pd.DataFrame([
#         ['aa','b','aab'],
#         ['a','b','ab'],
#     ],columns=['X','Y','Z']),
# ],
# )

In [ ]:
# TODO add tests for ie functions from stdlib, do this only for the ones we leave in the final demo version